<a href="https://colab.research.google.com/github/dasneelay/360Project/blob/main/HfPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup/Permissions

In [1]:
%pip install datasets torch transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 832.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl

In [ ]:
from huggingface_hub import notebook_login, Repository, get_full_repo_name

notebook_login()

In [3]:
import datasets
import torch
import transformers
import accelerate

# Loading Datasets
Load the train/valid datasets to create HF datasets object, then save it to Drive

In [6]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [7]:
# raw_datasets = datasets.load_dataset("dneelay/raw_datasets_mini")

# In case we need to clear the cache
datasets.load_dataset('dneelay/raw_datasets_mini', cache_dir=None, download_mode='force_redownload')

for feature in raw_datasets["train"][0]:
  print(f"{feature}: {raw_datasets['train'][0][feature][:200]}")

Generating train split: 0 examples [00:00, ? examples/s]

Failed to read file '/root/.cache/huggingface/datasets/downloads/7d1d8268c0b58900455df71886560d9414d3c39eb53cc95187ab86b2bcec2a26' with error <class 'datasets.table.CastError'>: Couldn't cast
repo_name: string
path: string
copies: string
size: string
content: string
license: string
-- schema metadata --
huggingface: '{"info": {"features": {"repo_name": {"dtype": "string", "_t' + 260
to
{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None)}
because column names don't match
ERROR:datasets.packaged_modules.arrow.arrow:Failed to read file '/root/.cache/huggingface/datasets/downloads/7d1d8268c0b58900455df71886560d9414d3c39eb53cc95187ab86b2bcec2a26' with error <class 'datasets.table.CastError'>: Couldn't cast
repo_name: string
path: string
copies: string
size: string
content: string
license: string
-- schema metadata --
huggingface: '{"info": {"features": {"repo_name": {"dtype": "string", "_t' + 260
to
{'input_ids': Sequence(feature=Value(dtype='int32', id=None), 

DatasetGenerationError: An error occurred while generating the dataset

In [ ]:
for key in raw_datasets["train"][0]:
    print(f"{key.upper()}: {raw_datasets['train'][0][key][:200]}")

REPO_NAME: Statoil/libecl
PATH: python/tests/ecl_tests/test_grid_pandas.py
COPIES: 2
SIZE: 4785
CONTENT: #!/usr/bin/env python
#  Copyright (C) 2018  Equinor ASA, Norway.
#
#  The file 'test_grid.pandas' is part of ERT - Ensemble based Reservoir Tool.
#
#  ERT is free software: you can redistribute it an
LICENSE: gpl-3.0


# Tokenization

In [ ]:
context_length = 128

tokenizer = transformers.AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")

outputs = tokenizer(
    raw_datasets["train"][:2]["content"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True
)
print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/789k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/448k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Input IDs length: 33
Input chunk lengths: [128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 24, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 103]
Chunk mapping: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

Chunk mapping: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
def tokenize(element):
    outputs = tokenizer(
        element["content"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True
    )

    input_batch = []

    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)

    return {"input_ids": input_batch}


# Training with Accelerate

In [ ]:
keytoken_ids = []
for keyword in [
    "plt",
    "pd",
    "sk",
    "fit",
    "predict",
    " plt",
    " pd",
    " sk",
    " fit",
    " predict",
    "testtest",
]:
    ids = tokenizer([keyword]).input_ids[0]
    if len(ids) == 1:
        keytoken_ids.append(ids[0])
    else:
        print(f"Keyword has not single token: {keyword}")

Keyword has not single token: testtest


In [ ]:
from torch.nn import CrossEntropyLoss
import torch


def keytoken_weighted_loss(inputs, logits, keytoken_ids, alpha=1.0):
    # Shift so that tokens < n predict n
    shift_labels = inputs[..., 1:].contiguous()
    shift_logits = logits[..., :-1, :].contiguous()
    # Calculate per-token loss
    loss_fct = CrossEntropyLoss(reduce=False)
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    # Resize and average loss per sample
    loss_per_sample = loss.view(shift_logits.size(0), shift_logits.size(1)).mean(axis=1)
    # Calculate and scale weighting
    weights = torch.stack([(inputs == kt).float() for kt in keytoken_ids]).sum(
        axis=[0, 2]
    )
    weights = alpha * (1.0 + weights)
    # Calculate weighted average
    weighted_loss = (loss_per_sample * weights).mean()
    return weighted_loss

In [ ]:
from torch.utils.data.dataloader import DataLoader

tokenized_datasets = datasets.load_dataset("dneelay/tokenized_datasets_mini")
tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=16, shuffle=True)
eval_dataloader = DataLoader(tokenized_datasets["valid"], batch_size=16)

In [ ]:
# Simplified script to debug dataloader initialization
print("Starting debugging script")

try:
    # Print the length of train_dataloader
    print(f"Length of train_dataloader: {len(train_dataloader)}")

    # Print a sample batch
    sample_batch = next(iter(train_dataloader))
    print(f"Sample batch: {sample_batch}")

except Exception as e:
    print(f"Error: {e}")

Starting debugging script
Length of train_dataloader: 86603
Sample batch: {'input_ids': tensor([[  312,   705,   380,  ...,   179,  2837,   547],
        [  262,    14,   322,  ...,   451,   396,   350],
        [   17,  1460,   222,  ...,  4447,  5358, 23944],
        ...,
        [   12,  1394,    12,  ...,    17,    61,   232],
        [  392,   269,  9808,  ...,  1332,   415,   869],
        [ 1623,   241, 28392,  ...,     8,  6417,    63]])}


In [ ]:
weight_decay = 0.1


def get_grouped_params(model, no_decay=["bias", "LayerNorm.weight"]):
    params_with_wd, params_without_wd = [], []
    for n, p in model.named_parameters():
        if any(nd in n for nd in no_decay):
            params_without_wd.append(p)
        else:
            params_with_wd.append(p)
    return [
        {"params": params_with_wd, "weight_decay": weight_decay},
        {"params": params_without_wd, "weight_decay": 0.0},
    ]

In [ ]:
def evaluate():
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch["input_ids"], labels=batch["input_ids"])

        losses.append(accelerator.gather(outputs.loss))
    loss = torch.mean(torch.cat(losses))
    try:
        perplexity = torch.exp(loss)
    except OverflowError:
        perplexity = float("inf")
    return loss.item(), perplexity.item()

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model = GPT2LMHeadModel(config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(get_grouped_params(model), lr=5e-4)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator(mixed_precision="fp16")

model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 1
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=1_000,
    num_training_steps=num_training_steps,
)

In [ ]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "codeparrot-ds-accelerate-save_pretrained"
repo_name = get_full_repo_name(model_name)
repo_name

'dneelay/codeparrot-ds-accelerate-save_pretrained'

In [ ]:
output_dir = "codeparrot-ds-accelerate-save_pretrained"
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/dneelay/codeparrot-ds-accelerate-save_pretrained into local empty directory.


In [ ]:
# # Remove the existing directory if it exists
# shutil.rmtree(output_dir, ignore_errors=True)

# # Re-clone the repository
# repo = Repository(output_dir, clone_from=repo_name)

## Checkpointing

### Starting Training Loop!!

In [ ]:
evaluate()

RuntimeError: zero-dimensional tensor (at position 0) cannot be concatenated

In [ ]:
from tqdm.notebook import tqdm

gradient_accumulation_steps = 8
batch_size = 32
samples_per_step = batch_size * gradient_accumulation_steps
eval_steps = 5_000
save_steps = 100

completed_steps = 0

# accelerator.print(
#     "Starting from checkpoint:\n"
#     f"\tstart_epoch: {start_epoch}\n"
#     f"\tstart_step: {start_step}\n"
#     f"\tcompleted_steps: {completed_steps}"
# )

model.train()


for epoch in range(num_train_epochs):
    for step, batch in tqdm(
        enumerate(train_dataloader, start=1), total=num_training_steps
    ):
        logits = model(batch["input_ids"]).logits
        loss = keytoken_weighted_loss(batch["input_ids"], logits, keytoken_ids)
        if step % 100 == 0:
            accelerator.print(
                {
                    "samples": step * samples_per_step,
                    "steps": completed_steps,
                    "loss/train": loss.item() * gradient_accumulation_steps,
                }
            )
        loss = loss / gradient_accumulation_steps
        accelerator.backward(loss)
        if step % gradient_accumulation_steps == 0:
            accelerator.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            completed_steps += 1

        if (step % (eval_steps * gradient_accumulation_steps//4)) == 0:
            eval_loss, perplexity = evaluate()
            accelerator.print({"loss/eval": eval_loss, "perplexity": perplexity})
            model.train()
            accelerator.wait_for_everyone()
            unwrapped_model = accelerator.unwrap_model(model)
            unwrapped_model.save_pretrained(output_dir, saave_function=accelerator.save)
            if accelerator.is_main_process:
              tokenizer.save_pretrained(output_dir)
              repo.push_to_hub(
                  commit_message = f"Training in progress step {step}", blocking=False
              )


  0%|          | 0/43302 [00:00<?, ?it/s]

{'samples': 25600, 'steps': 12, 'loss/train': 75.9399642944336}
{'samples': 51200, 'steps': 24, 'loss/train': 61.610382080078125}
{'samples': 76800, 'steps': 37, 'loss/train': 75.51673889160156}
{'samples': 102400, 'steps': 49, 'loss/train': 53.90418243408203}
{'samples': 128000, 'steps': 62, 'loss/train': 75.63220977783203}
{'samples': 153600, 'steps': 74, 'loss/train': 69.59214782714844}
{'samples': 179200, 'steps': 87, 'loss/train': 55.82472229003906}
{'samples': 204800, 'steps': 99, 'loss/train': 48.54944610595703}
{'samples': 230400, 'steps': 112, 'loss/train': 65.49447631835938}
{'samples': 256000, 'steps': 124, 'loss/train': 55.913177490234375}
{'samples': 281600, 'steps': 137, 'loss/train': 74.5710678100586}
{'samples': 307200, 'steps': 149, 'loss/train': 56.81859588623047}
{'samples': 332800, 'steps': 162, 'loss/train': 63.593971252441406}
{'samples': 358400, 'steps': 174, 'loss/train': 43.64073944091797}
{'samples': 384000, 'steps': 187, 'loss/train': 56.407470703125}
{'sampl

### Resuming Training Loop!!

# Test with Pipeline

In [ ]:
import torch
from transformers import pipeline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
pipe = pipeline(
    "text-generation", model="dneelay/codeparrot-ds", device=device
)

config.json:   0%|          | 0.00/902 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/497M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/789k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/448k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

In [ ]:
txt = """\
# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create scatter plot with x, y
"""
print(pipe(txt, num_return_sequences=1)[0]["generated_text"])

# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create scatter plot with x, y

# create a contour plot
plt.subplot(221)


In [ ]:
txt = """\
# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create dataframe from x and y
"""
print(pipe(txt, num_return_sequences=1)[0]["generated_text"])

# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create dataframe from x and y
data = [[1, 2, 3], [4, 5,


NameError: name 'evaluate' is not defined